In [ ]:
# using CuArrays
using AutoGrad
using Deeplearning
using Statistics
using StatsBase
using Plots
import CSV

In [ ]:
batch = 100
learning_rate = 10e-4

In [ ]:
train = CSV.read("datasets/digit-recognizer/train.csv")
xtrain = []
ytrain = []

for i in 1:size(train)[1]
    max = maximum(train[i,2:end])+0.1
    normalized = reshape(train[i, 2:end]./(max), (28,28,1,1))
    push!(xtrain, normalized)
    push!(ytrain, train[i,1]+1)
end

println("Ready...")

In [ ]:
mnist_train = Deeplearning.minibatch(xtrain, ytrain, batch, shuffle=true)

In [ ]:
conv1 = Deeplearning.ConvolutionLayer((3,3,6), (2,2), (1,1), xavier, relu)((28,28,1,1))
conv2 = Deeplearning.ConvolutionLayer((3,3,16), (2,2), (1,1), xavier, relu)(conv1.telemetry.o)
dense1 = Deeplearning.FullyConnectedLayer(120, xavier, relu)(conv2.telemetry.o)
dense2 = Deeplearning.FullyConnectedLayer(84, xavier, relu)(dense1.telemetry.o)
dense3 = Deeplearning.FullyConnectedLayer(10, xavier, relu)(dense2.telemetry.o)
layers=[dense3,dense2,dense1,conv2,conv1]
println("Loaded")

In [ ]:
function f(x)
    x = conv1(x,s=(2,2))    
    x = conv2(x,s=(2,2))    
    x = dense1(x)
    x = dense2(x)
    x = dense3(x)
    x = softmax(x)
end

In [ ]:
lenet5v2 = Deeplearning.Network(layers, f, loss=nll)

indices = sample(1:size(train)[1], size(train)[1], replace = false)
println("Ready...")

sample_batch_x, sample_batch_y = cat(xtrain[indices[(1):(100)]]..., dims=4), ytrain[indices[(1):(100)]]
println("Ready...")

using BenchmarkTools

dv = dff(sample_batch_x, sample_batch_y)

dvp = @parameters dv

In [ ]:
acc_train_plot = []
acc_test_plot = []
loss_train_plot = []
loss_test_plot = []

In [ ]:
datastream = Deeplearning.progress(repeat(mnist_train, 1))

In [ ]:
di = collect(Iterators.take(mnist_train, 1))

In [ ]:
batch_x, batch_y = di[1]

In [ ]:
dv = @diff lenet5v2(cat(batch_x...,dims=4), batch_y)

In [ ]:
dparams = @parameters dv

In [ ]:
dw = grad(dv, dparams[1])

In [ ]:
for (iteration, data) in enumerate(datastream)
    batch_x, batch_y = data

    dv = @diff lenet5v2(cat(batch_x...,dims=4), batch_y)#dff(sample_batch_x, sample_batch_y)

    dparams = @parameters dv

    for pidx in 1:length(dparams[1:end-1])
        dw = grad(dv, dparams[pidx])
        dw .-=learning_rate*dw
    end
    
#     if(iteration%sample_int==1)
        
#     end
    
end

In [ ]:
for e in 1:1
    println("Epoch ", e)
    indices = sample(1:size(train)[1], size(train)[1], replace = false)
    for (ei,si) in enumerate(1:batch:size(indices)[1])
        sample_batch_x, sample_batch_y = cat(xtrain[indices[(si):(ei*batch)]]..., dims=4), ytrain[indices[(si):(ei*batch)]]

        dv = dff(sample_batch_x, sample_batch_y)

        dparams = @parameters dv

        for pidx in 1:length(dparams[1:end-1])
            dw = grad(dv, dparams[pidx])
            dw .-=learning_rate*dw
        end
    end
    otrain = f(xtrain)
#     otest = f(xtest)
    push!(acc_train_plot, acc(otrain, ytrain))
    push!(loss_train_plot, nll(otrain, ytrain))
#     push!(acc_test_plot, acc(o, ytest))
#     push!(loss_test_plot, nll(o, ytest))
end

In [ ]:
plot([acc_plot], linewidth=2, title="Accuracy")

In [ ]:
plot([loss_plot], linewidth=2, title="Loss")